Project by Jeremy Bouhi & Lucas Trevalinet

# Classification Document


## Imports

In [2]:
import numpy as np
import pandas as pd
import os
import xml.etree.ElementTree as ET
from pathlib import Path
import re
import pyprind

## STEP 1 : Grab Data

In [22]:
# We consider you had already unzip the dataset file and that this file is in the parent folder

#GLOBAL_PATH = os.path.join("Freemium_cass_global_20180315-170000","20180315-170000", "juri","cass","global") #Lucas' Path
GLOBAL_PATH = os.path.join("..","20180315-170000", "juri","cass","global") #Jeremy's Path

# We need to do that for excluding data from Juri_path
CIVILE_PATH = os.path.join(GLOBAL_PATH,"civile")
COMMERCIALE_PATH = os.path.join(GLOBAL_PATH,"commerciale")
CRIMINELLE_PATH = os.path.join(GLOBAL_PATH,"criminelle")
SOCIALE_PATH = os.path.join(GLOBAL_PATH,"sociale")

In [63]:
from nltk.tokenize import sent_tokenize

ident = []
text = []
division = []
#DATA_PATH = [SOCIALE_PATH] # For faster test
DATA_PATH = [CIVILE_PATH, COMMERCIALE_PATH, CRIMINELLE_PATH, SOCIALE_PATH]

for DIVISION_PATH in DATA_PATH :
    xml_files = list(Path(DIVISION_PATH).glob('**/*.xml'))
    
    for xml_file in xml_files:
    
        with open(xml_file, 'r', encoding="utf-8") as content:

            etree = ET.parse(content) #create an ElementTree object 
            root = etree.getroot()
            
            # For getting the ID
            for child in root.iter('META_COMMUN'):
                id = child.find('ID').text
                ident.append(id)

            
            for child in root.iter('BLOC_TEXTUEL'):
                contenu = "".join(child.itertext())
                text.append(contenu)
                
            # For getting the division    
            for child in root.iter('META_JURI_JUDI'):
                formation = re.sub('CHAMBRE|_|[0-9]', '', child.find('FORMATION').text)
                division.append(formation)


## STEP 2 : Create DataFrame to manipulate easily data

In [104]:
d = {'id': ident, 'text': text, 'division': division}
df = pd.DataFrame(data = d)
    
# remove all NaN values from df
df.dropna()

# save for the next time
df.to_pickle('3FD8KA7.pkl')
    

MemoryError: 

In [3]:
df = pd.read_pickle('3FD8KA7.pkl')

In [105]:
fast_execution = True #to run code faster
mini_df = df.sample(500)

if(fast_execution) : 
    X = mini_df['text']
    y = mini_df['division']
    print('You chose to run using fast_execution')
    print('data size: ', X.shape[0])
else :
    X = df['text']
    y = df['division']
    print('You chose to run with the whole dataset')
    print('data size: ', X.shape[0])

You chose to run using fast_execution
data size:  500


## STEP 4 : Clean Data

In [106]:
def clean_references_to_law_codes(text) : 
    return re.sub('(?<=Code )civil|de l\'action sociale et des familles|de l\'artisanat|des assurances|de l\'aviation civile|du cinéma et de l image animée|de commerce|des communes( de la Nouvelle-Calédonie)?|de la consommation', '', ''.join(text))


X = X.apply(clean_references_to_law_codes)

In [114]:
def clean_references_to_law_articles(text) :
    return re.sub(r'((?:\b[A-Z][\.-]? ?)?(?:\d+-?\d+\b))', 'N', ''.join(text))

X = X.apply(clean_references_to_law_articles)

In [108]:
# Preprocessing step pull off all the htlm tag, "\n", and any special characters
def preprocessor(text):
    text = re.sub('<[^>]*>', '', ''.join(text)) # remove HTML tags
    text = re.sub('[,\/#!$%\^&\*:{}=\_`~()«»°–]','', ''.join(text)) # remove special characters except ; and . for spliting in sentences
    text = re.sub('\.\.\.','', ''.join(text)) # remove ...
    text = re.sub('\bM\. \b','M ', ''.join(text)) # explicit replacement of M. (because of the .)
    text = text.replace(';','.') # some sentences are split by ;
    text = text.replace('\n','').replace('\'',' ')
    text.replace('\s{2,}',' ')
    return text

X = X.apply(preprocessor)

In [109]:
# Set to lower case
def to_lower_case(s) :
    return s.lower()

X = X.apply(to_lower_case)

In [110]:
# We decided to replace any corresponding labels to a unique special label : *_* 
def clean_words_corresponding_to_labels(text) :
    return re.sub('criminel(le)?|commercial(e)?|social(e)?|civil(e)?', '*_*', ''.join(text))

X = X.apply(clean_words_corresponding_to_labels)

In [111]:
#Stop_words :
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

def remove_stop_words(text):
    
    french_stopwords = set(stopwords.words('french'))
    tokens = word_tokenize(text, language='french')
    
    content_tokens = ""
    for token in tokens:
        if token not in french_stopwords:
            content_tokens += token
            content_tokens += " "
    return(content_tokens) 

X = X.apply(remove_stop_words)

In [49]:
text1 = "coucou c'est moi les amis je suis super content étais etait" # 

In [50]:
remove_stop_words(text)

"coucou c'est les amis super content etait "

## Most influential words

In [46]:
X_civile = X.where(y=='CIVILE')
X_commerciale = X.where(y=='COMMERCIALE')
X_criminelle = X.where(y=='CRIMINELLE')
X_sociale = X.where(y=='SOCIALE')

#to remove NaN values
X_civile = X_civile[~X_civile.isnull()]
X_commerciale = X_commerciale[~X_commerciale.isnull()]
X_criminelle = X_criminelle[~X_criminelle.isnull()]
X_sociale = X_sociale[~X_sociale.isnull()]

In [47]:
#so they all have differents shapes
print(X_civile.shape[0])
print(X_commerciale.shape[0])
print(X_criminelle.shape[0])
print(X_sociale.shape[0])

241
74
76
109


In [48]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

def get_influential_words(X, top=10) :
    count_vect = CountVectorizer()
    count_vect = count_vect.fit(X)
    freq_term_matrix = count_vect.transform(X)

    tfidf = TfidfTransformer(norm="l2")
    tfidf.fit(freq_term_matrix)
    
    doc_freq_term = count_vect.transform(X)
    doc_tfidf_matrix = tfidf.transform(doc_freq_term)
    
    tfidf_sorting = np.argsort(doc_tfidf_matrix.toarray()).flatten()[::-1]
    feature_array = np.array(count_vect.get_feature_names())
    top_n = feature_array[tfidf_sorting][:top]
    
    return top_n

In [49]:
top_civile = get_influential_words(X_civile)
top_commerciale = get_influential_words(X_commerciale)
top_criminelle = get_influential_words(X_criminelle)
top_sociale = get_influential_words(X_sociale)

In [50]:
per_division = {'civile': top_civile, 'commerciale': top_commerciale, 'criminelle': top_criminelle, 'sociale': top_sociale}
words_df = pd.DataFrame(data = per_division)

words_df

,civile,commerciale,criminelle,sociale
0,salariee,privilege,somme,juillet
1,activite,traitant,francs,comptable
2,commerce,publics,commission,qualité
3,salaries,agrement,titre,cadre
4,registre,travaux,remise,convention
5,inscrit,marches,occulte,comptables
6,travailleurs,creancier,montant,stagiaire
7,decision,nanti,confiance,juin
8,regionale,sous,arret,troisième
9,exerce,cavaille,relaxe,salarié


## Build document summarizer

Once you'll build the tf-idf matrix for the text corpus, you will use the tf-idf of each word to compute a value for each sentence. The n top sentences for a document will be used to represent the document.

In [119]:
from nltk.tokenize import sent_tokenize

print(df['text'][52075])
print(X[52075])

sent_tokenize(X[52075])




   SUR LE MOYEN UNIQUE PRIS EN SES DEUX BRANCHES :
    ATTENDU QU'IL EST FAIT GRIEF A L'ARRET CONFIRMATIF ATTAQUE D'AVOIR DECLARE COMMERCIAL LE BAIL LITIGIEUX, CONSTATE L'ABSENCE D'UNE EXPLOITATION ARTISANALE OU COMMERCIALE ET LE CHANGEMENT DE DESTINATION DES LIEUX, VALIDE LE CONGE DELIVRE PAR DAME Y..., PROPRIETAIRE, A MATHIEU, PRENEUR, ET PRONONCE L'EXPULSION DE CELUI-CI, ALORS, SELON LE MOYEN, QUE, D'UNE PART, LEDIT PRENEUR AVAIT, DANS SES CONCLUSIONS D'APPEL DEMEUREES SANS REPONSE, SOULIGNE QUE L'ALLUSION AU CARACTERE COMMERCIAL D'UNE PARTIE DES LOCAUX ETAIT "SANS INCIDENCE" DES LORS QUE LE CONTRAT NE COMPORTAIT AUCUNE DES CLAUSES USUELLES D'UN BAIL COMMERCIAL, SPECIALEMENT LA NECESSITE D'EXPLOITER DANS LES LIEUX UN FONDS DE COMMERCE ET DE GARNIR LES LOCAUX LOUES DE MATERIEL ET DE MARCHANDISES SUFFISANTS POUR GARANTIR LE PAYEMENT DES LOYERS ET QUE, D'AUTRE PART, LA VOLONTE DES PARTIES DE CONCLURE UN BAIL COMMERCIAL "NE SUFFISANT PAS A LA LEGISLATION SUR LES BAUX COMMERCIAUX", LES 

["moyen unique pris deux branches attendu fait grief a arret confirmatif attaque avoir declare *_* bail litigieux constate absence exploitation artisanale *_* changement destination lieux valide conge delivre dame proprietaire a mathieu preneur prononce expulsion celui-ci alors selon moyen part ledit preneur conclusions appel demeurees sans reponse souligne allusion caractere *_* partie locaux etait `` sans incidence '' lors contrat comportait aucune clauses usuelles bail *_* specialement necessite exploiter les lieux fonds commerce garnir les locaux loues materiel marchandises suffisants garantir payement loyers autre part volonte parties conclure bail *_* `` suffisant a legislation les baux commerciaux '' les juges fond devaient rechercher quelles ete fait intention parties nature utilisation lieux .",
 'attendu caractere location determine non usage locataire a pu faire chose louee destination les parties entendu donner lors conclusion contrat .',
 "adoption motifs cour appel consta

In [68]:
count_vect = CountVectorizer()
count_vect = count_vect.fit(X)
freq_term_matrix = count_vect.transform(X)

tfidf = TfidfTransformer(norm="l2")
tfidf.fit(freq_term_matrix)
    
doc_freq_term = count_vect.transform(X)
doc_tfidf_matrix = tfidf.transform(doc_freq_term)

['\nSUR LE PREMIER MOYEN, PRIS DE LA VIOLATION DES ARTICLES 541 DU CODE DE PROCEDURE CIVILE, 24-A DU LIVRE 1ER DU CODE DU TRAVAIL ET 7 DE LA LOI DU 20 AVRIL 1810, DEFAUT DE MOTIFS, DEFAUT DE REPONSE AUX CONCLUSIONS, MANQUE DE BASE LEGALE ;\n\nATTENDU QU\'IL RESULTE DU JUGEMENT ATTAQUE QUE OTT, AU SERVICE DE LA COMPAGNIE INTERNATIONALE DES WAGONS-LITS DEPUIS LE 12 MARS 1959, ARRETA SON TRAVAIL POUR MALADIE LE 7 FEVRIER 1968 ;\n\n QU\'EXAMINE PAR LE MEDECIN DE L\'ENTREPRISE LE 7 MARS SUIVANT IL FUT DECLARE DEFINITIVEMENT INAPTE A SON EMPLOI ;\n\n QUE LE MEME JOUR LA COMPAGNIE INFORMA OTT QUE LE CONTRAT DE TRAVAIL SE TROUVAIT ROMPU ET QUE SES SERVICES PRENAIENT FIN LE 6 MARS 1968 AU SOIR ;\n\n QUE LES 15 MARS ET 9 AVRIL 1968 OTT SIGNA DEUX "RECUS POUR SOLDE DE TOUS COMPTES", L\'UN POUR LES SALAIRES ET INDEMNITES AFFERENTES A LA PERIODE DU 1ER AU 6 MARS 1968, L\'AUTRE POUR L\'INDEMNITE DE CONGEDIEMENT CALCULEE SELON LA CONVENTION COLLECTIVE APPLICABLE ;\n\n QUE OTT ASSIGNA LA COMPAGNIE EN 

## STEP 3 : Split Data

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test= train_test_split(X, y, test_size = 0.2 ,random_state=42,stratify=df["division"] )
print(X_train.head())

## STEP 5 : Train the model

In [21]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [22]:
rnd_clf = RandomForestClassifier(random_state=42, n_estimators=100)
rnd_clf.fit(X_train, y_train)
y_pred = rnd_clf.predict(X_test)

ValueError: could not convert string to float: "\nSur le moyen unique du pourvoi incident, qui est préalable :\n   Attendu, selon l'arrêt attaqué qu'un tribunal de grande instance a prononcé la séparation de corps des époux X... aux torts de l'épouse ; que, par assignation du 21 juillet 1989, le mari a assigné sa femme aux fins de conversion de la séparation de corps en divorce ; que, par assignation en date du 2 août 1989, Mme X..., autorisée par une ordonnance de non-conciliation du 19 janvier 1989, a assigné son mari en divorce pour faute ; qu'après jonction de ces deux procédures, le tribunal de grande instance a prononcé la conversion de la séparation de corps en divorce aux torts de la femme, et, accueillant la demande en divorce de celle-ci, prononcé également le divorce à son profit ; qu'ensuite le Tribunal a constaté que les époux étaient ainsi divorcés aux torts partagés ;\n   Attendu qu'il est fait grief à l'arrêt infirmatif attaqué d'avoir déclaré irrecevable la demande en 

In [153]:
rnd_clf = RandomForestClassifier(random_state=42, n_estimators=100)

ada_clf= AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1), n_estimators=100,
    algorithm="SAMME.R", learning_rate=0.5, random_state=42)

gdb_clf = GradientBoostingClassifier(random_state=42, n_estimators=100)

voting_clf = VotingClassifier(
    estimators=[('rfc', rnd_clf), ('abc', ada_clf), ('gbr', gdb_clf)],
    voting='hard')

In [154]:
for clf in (rnd_clf, ada_clf, gdb_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

ValueError: could not convert string to float: 'sur le moyen unique du pourvoi incident qui est préalable    attendu selon l arrêt attaqué qu un tribunal de grande instance a prononcé la séparation de corps des époux x aux torts de l épouse  que par assignation du 21 juillet 1989 le mari a assigné sa femme aux fins de conversion de la séparation de corps en divorce  que par assignation en date du 2 août 1989 mme x autorisée par une ordonnance de nonconciliation du 19 janvier 1989 a assigné son mari en divorce pour faute  qu après jonction de ces deux procédures le tribunal de grande instance a prononcé la conversion de la séparation de corps en divorce aux torts de la femme et accueillant la demande en divorce de celleci prononcé également le divorce à son profit  qu ensuite le tribunal a constaté que les époux étaient ainsi divorcés aux torts partagés    attendu qu il est fait grief à l arrêt infirmatif attaqué d avoir déclaré irrecevable la demande en divorce de la femme alors que d 